# the input data source is IB statement OR Toms daily trade blotter, the output is csv report, pnl logic is first in first out 

In [38]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
from collections import deque
from blpapi import Session, SessionOptions
from blp import blp
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import math
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows

# IB

In [9]:
# read the data from ib statement file
ib_data = []
with open('U12393385_20250101_20250206.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        ib_data.append(row)

In [10]:
def convert_ib_code_to_bb_ticker(row):
    """manually maintain a mapping of IB codes to Bloomberg tickers"""
    if row['Asset Category'] == 'Stocks' and row['Currency'] == 'USD':
        return row['Symbol'] + ' US Equity'
    # need change
    # if row['Asset Category'] == 'Futures':
    #     return row['Symbol'] + ' Comdty'
    pass

In [11]:
# data cleaning
trades = []
for row in ib_data:
    if row[0] == 'Trades':
        trades.append(row)
# crete dataframe from trades
df = pd.DataFrame(trades[1:], columns=trades[0])
df['BB_Symbol'] = df.apply(convert_ib_code_to_bb_ticker, axis=1)
df.drop(columns=['Trades', 'Header', 'DataDiscriminator','Realized P/L','MTM P/L','Code'], inplace=True)
stock_df = df[df['Asset Category'] == 'Stocks']
futures_df = df[df['Asset Category'] == 'Futures']
#drop the rows that have no value in the Date/Time column
stock_df = stock_df[stock_df['Date/Time'] != '']
stock_df['Date/Time'] = pd.to_datetime(stock_df['Date/Time'])
futures_df = futures_df[futures_df['Date/Time'] != '']
futures_df['Date/Time'] = pd.to_datetime(futures_df['Date/Time'])
columns_to_convert = ['Quantity', 'T. Price', 'C. Price','Comm/Fee']
for col in columns_to_convert:
    stock_df[col] = pd.to_numeric(stock_df[col].str.replace(',', ''))


In [12]:
stock_df

,Asset Category,Currency,Symbol,Date/Time,Quantity,T. Price,C. Price,Proceeds,Comm/Fee,Basis,BB_Symbol
0,Stocks,USD,SGOV,2025-01-08 05:37:10,15184,100.41,100.41,-1524625.44,-102.303301,1524727.743300841,SGOV US Equity
1,Stocks,USD,SGOV,2025-01-31 11:41:08,-8996,100.69,100.68,905807.24,-45.751379,-902569.3436,SGOV US Equity
2,Stocks,USD,SGOV,2025-01-31 14:52:27,-23576,100.69,100.68,2373867.44,-108.514540,-2365390.774784,SGOV US Equity
3,Stocks,USD,SGOV,2025-01-31 19:46:02,-37742,100.68,100.68,3799864.56,-366.190868,-3786594.285461001,SGOV US Equity
4,Stocks,USD,SGOV,2025-02-04 10:55:28,46715,100.36,100.36,-4688317.4,-101.697699,4688419.097699339,SGOV US Equity
5,Stocks,USD,SGOV,2025-02-04 10:55:29,23815,100.36,100.36,-2390073.4,-82.007289,2390155.407289088,SGOV US Equity


In [3]:
# blb test
bquery = blp.BlpQuery().start() 
df = bquery.bdh(["SGOV US Equity"], ["PX_LAST"], "20250214", "20250220")[['date', 'PX_LAST']]
df

,date,PX_LAST
0,2025-02-14,100.50
1,2025-02-18,100.52
2,2025-02-19,100.52


In [14]:
def get_close_price_from_bloomberg(ticker, query_date):
    """
    eg:'SGOV US Equity','2025-02-11'
    when query_date is not specified, treat it as the latest close price
    """
    if query_date is None:
        query_date = datetime.now()
    else:
        query_date = datetime.strptime(query_date, '%Y-%m-%d')
    if 'US Equity' in ticker:
        if query_date.weekday() == 0:
            query_date = query_date - timedelta(days=3)
        else:
            query_date = query_date - timedelta(days=1) 
    else: # Asia market
        if query_date.time() < datetime.strptime('17:00:00', '%H:%M:%S').time():
            query_date = query_date - timedelta(days=1)
    query_date = query_date.strftime('%Y%m%d')
            
    bquery = blp.BlpQuery().start()
    market_price = float('nan')  
    try:
        df = bquery.bdh([ticker], ['PX_LAST'], query_date, query_date)[['date', 'PX_LAST']]
        if df.empty:
            print(f"{ticker} at {query_date} not found!!")
        else:
            market_price = df['PX_LAST'].values[0]
    except Exception as e:
        print(f"{ticker} at {query_date} not found!!")
        
    finally:
        bquery.stop()
        
    return market_price

In [15]:
class TradingPosition:
    '''define a class to record each trade and calculate pnl'''
    def __init__(self):
        self.position = deque()  
        self.realized_pnl = 0

    def make_trade_record(self, price, quantity, fee):
        # fee is not divided，it represents the cost at the trade happens
        self.position.append((price, quantity, fee))  
        self.realized_pnl += fee    
        
    def get_realized_pnl(self, price, quantity, fee):
        # get realized pnl when position closed 
        # handle both long and short
        # this function is used for offsetting the position
        unclosed_quantity = quantity
        
        if self.position:
            if self.position[0][1]>0: # if position is long
                while (unclosed_quantity < 0 and self.position): #  when selling and still have positive position
                    buy_price, buy_quantity, buy_fee = self.position[0] # get previous buy price
                    if buy_quantity > (- unclosed_quantity): # when have enough position
                        self.realized_pnl += (price - buy_price) * (- unclosed_quantity) 
                        print('realized_pnl1',self.realized_pnl)
                        # offset to 1 after selling
                        self.position.popleft()
                        self.position.pop()
                        self.position.appendleft((buy_price, buy_quantity + unclosed_quantity, buy_fee))
                        unclosed_quantity = 0
                        print("HERE1",self.position)
                    else: # when don't have enough position
                        self.realized_pnl += (price - buy_price) * buy_quantity 
                        print('realized_pnl2',self.realized_pnl)
                        unclosed_quantity += buy_quantity
                        # offset 
                        self.position.popleft() 
                        self.position.pop()
                        print("HERE2",self.position)
                        if(unclosed_quantity < 0 and self.position):
                            self.position.append((price, unclosed_quantity, fee))  
                # add new position after offsetting
                if unclosed_quantity < 0 and not self.position:
                    self.position.append((price, unclosed_quantity, fee))  
            else: # if position is short
                while (unclosed_quantity > 0 and self.position): #when buying and still have position
                    sell_price, sell_quantity, sell_fee = self.position[0] # get previous selling price
                    if (- sell_quantity) > unclosed_quantity: # when have enough position
                        self.realized_pnl += (sell_price - price) * unclosed_quantity 
                        print('realized_pnl3',self.realized_pnl)
                        self.position.popleft()
                        self.position.pop()
                        self.position.appendleft((sell_price, sell_quantity + unclosed_quantity, sell_fee))
                        unclosed_quantity = 0
                        print("HERE3",self.position)
                    else:  # when don't have enough position
                        self.realized_pnl += (sell_price - price) * (- sell_quantity) 
                        print('realized_pnl4',self.realized_pnl)
                        unclosed_quantity += sell_quantity
                        # offset
                        self.position.popleft() 
                        self.position.pop()
                        print("HERE4",self.position)
                        if(unclosed_quantity > 0 and self.position):
                            self.position.append((price, unclosed_quantity, fee))
                if unclosed_quantity > 0: # add new position after offsetting
                    self.position.append((price, unclosed_quantity, fee))
                                                    
        print("HERE5",self.position)
        print("realized_pnl",self.realized_pnl)     
        print("----------------------------------------")
        
        return self.realized_pnl
    
    def get_unrealized_pnl(self, current_price):
        unrealized_pnl = 0
        if self.position:
            for price, quantity, fee in self.position:
                unrealized_pnl += (current_price - price) * quantity
        else:
            pass
        return float(unrealized_pnl)
    
    def current_position_df(self):
        df = pd.DataFrame(list(self.position), columns=['price, quantity, fee'])
        return df

In [16]:
def calculation(df, start_date, end_date):
    '''calculate pnl, exposure and return in a period, if date is not specified, treat it as the whole period.'''
    trading_positions = {}  
    realized_pnl = 0
    unrealized_pnl = 0
    
    df = df[(df['Date/Time'] >= start_date) & (df['Date/Time'] <= end_date)]
    df_sorted = df.sort_values(by=['BB_Symbol', 'Date/Time'])
    for bb_symbol, group in df_sorted.groupby('BB_Symbol'):
        if bb_symbol not in trading_positions:
            trading_positions[bb_symbol] = TradingPosition()
    
        for idx, row in group.iterrows():
            quantity = row['Quantity']
            price = row['T. Price']
            fee = row['Comm/Fee']
            trading_positions[bb_symbol].make_trade_record(price, quantity, fee)
            realized_pnl = trading_positions[bb_symbol].get_realized_pnl(price, quantity, fee)
        
    # calculate unrealized pnl based on current position  
    for bb_symbol in trading_positions.keys():
        current_price = get_close_price_from_bloomberg(bb_symbol, end_date)
        unrealized_pnl = trading_positions[bb_symbol].get_unrealized_pnl(current_price)
    
    return realized_pnl, unrealized_pnl



In [17]:
realized_pnl, unrealized_pnl = calculation(stock_df, '2025-01-01','2025-02-14')

HERE5 deque([(100.41, 15184, -102.303300841)])
realized_pnl -102.303300841
----------------------------------------
realized_pnl1 2370.82531967501
HERE1 deque([(100.41, 6188, -102.303300841)])
HERE5 deque([(100.41, 6188, -102.303300841)])
realized_pnl 2370.82531967501
----------------------------------------
realized_pnl2 3994.9507795830173
HERE2 deque([])
HERE5 deque([(100.69, -17388, -108.514540092)])
realized_pnl 3994.9507795830173
----------------------------------------
HERE5 deque([(100.69, -17388, -108.514540092), (100.68, -37742, -366.190868061)])
realized_pnl 3628.7599115220173
----------------------------------------
realized_pnl4 9265.102212183987
HERE4 deque([(100.68, -37742, -366.190868061)])
realized_pnl3 18649.742212184203
HERE3 deque([(100.68, -8415, -366.190868061)])
HERE5 deque([(100.68, -8415, -366.190868061)])
realized_pnl 18649.742212184203
----------------------------------------
realized_pnl4 21260.534923096264
HERE4 deque([])
HERE5 deque([(100.36, 15400, -82.007

In [18]:
realized_pnl, unrealized_pnl

(21260.534923096264, 1539.9999999999125)

In [19]:
# WTD
last_wtd_end_pnl = calculation(stock_df, '2025-01-01', '2025-02-14')
wtd_pnl = realized_pnl-last_wtd_end_pnl[0], unrealized_pnl-last_wtd_end_pnl[1]
# MTD
last_mtd_end_pnl = calculation(stock_df, '2025-01-01', '2025-01-31')
mtd_pnl = realized_pnl-last_mtd_end_pnl[0], unrealized_pnl-last_mtd_end_pnl[1]
# YTD
last_ytd_end_pnl = calculation(stock_df, '2024-01-01', '2024-12-31')
ytd_pnl = realized_pnl-last_ytd_end_pnl[0], unrealized_pnl-last_ytd_end_pnl[1]

HERE5 deque([(100.41, 15184, -102.303300841)])
realized_pnl -102.303300841
----------------------------------------
realized_pnl1 2370.82531967501
HERE1 deque([(100.41, 6188, -102.303300841)])
HERE5 deque([(100.41, 6188, -102.303300841)])
realized_pnl 2370.82531967501
----------------------------------------
realized_pnl2 3994.9507795830173
HERE2 deque([])
HERE5 deque([(100.69, -17388, -108.514540092)])
realized_pnl 3994.9507795830173
----------------------------------------
HERE5 deque([(100.69, -17388, -108.514540092), (100.68, -37742, -366.190868061)])
realized_pnl 3628.7599115220173
----------------------------------------
realized_pnl4 9265.102212183987
HERE4 deque([(100.68, -37742, -366.190868061)])
realized_pnl3 18649.742212184203
HERE3 deque([(100.68, -8415, -366.190868061)])
HERE5 deque([(100.68, -8415, -366.190868061)])
realized_pnl 18649.742212184203
----------------------------------------
realized_pnl4 21260.534923096264
HERE4 deque([])
HERE5 deque([(100.36, 15400, -82.007

In [20]:
# output in csv file
with open('pnl.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['PnL', 'WTD', 'MTD', 'YTD'])
    writer.writerow(['Realized', realized_pnl, wtd_pnl[0], mtd_pnl[0], ytd_pnl[0]])
    writer.writerow(['Unrealized', unrealized_pnl, wtd_pnl[1], mtd_pnl[1], ytd_pnl[1]])

# Toms Blotter

In [30]:
def traditional_round(num, n):
    multiplier = 10 ** n
    return int(num * multiplier + 0.5) / multiplier

In [ ]:
def read_txt_to_dataframe(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        # header start with 'Trader'
        header_index = next(i for i, line in enumerate(lines) if line.startswith('Trader'))
        headers =['Trader','Counterpar','B/S','Long Description','ISIN','Amount','Trd Dt','As of Dt','Transac','Trm Date','Stl Date','Coupon','Crcy','Price','Principal','Accr Int','Settlement Amount','Repo Rte Haircut','Unadj Term Money','Tkt #','Execution Order Identifier','Broker Commissi','Stamp Duty Amou','Transaction Lev','Exchange Fee Am','Miscellaneous F','Commission']
        # get the index of each item in headers
        headers_index = [lines[header_index].index(header) for header in headers]
        headers_index.append(len(lines[header_index]))
        for line in lines:
            if line.startswith('-') or line.startswith(' ') or line.startswith('START OF REPORT') or line.startswith('END OF REPORT') or line.startswith('Trader'):
                continue
            # split the line by index in headers_index
            values = [line[headers_index[i]:headers_index[i+1]].strip() for i in range(len(headers_index)-1)]
            data.append(values)
    
    # create dataframe and do basic cleaning
    df = pd.DataFrame(data, columns=headers)
    df = df.replace('', np.nan)
    df = df.dropna(axis=0,how='all')
    df.reset_index(drop=True, inplace=True)
    df['Amount'] = pd.to_numeric(df['Amount'].str.replace(',', ''))
    df['Price'] = pd.to_numeric(df['Price'])
    df['Long Description'] = pd.to_numeric(df['Long Description'])
    # don't rely on the existing Principal column, recalculate it
    df['Principal'] = df['Amount']*df['Price']
    #drop columns
    df.drop(columns=['Broker Commissi','Stamp Duty Amou','Transaction Lev','Exchange Fee Am','Miscellaneous F','Commission'], inplace=True)
    # if Execution Order Identifier is NA, fill it with a random number, don't repeat
    for i in range(len(df)):
        if pd.isna(df.loc[i, 'Execution Order Identifier']):
            df.loc[i, 'Execution Order Identifier'] = hash(str(i))
    return df

In [32]:
file_path = 'toms_trade_blotter_20250213.txt' 
df = read_txt_to_dataframe(file_path)
df.columns
# count distinct values in each column
# TKT # is the primary key
# Execution Order Identifier is mather order id, if the same, then should merge

Index(['Trader', 'Counterpar', 'B/S', 'Long Description', 'ISIN', 'Amount',
       'Trd Dt', 'As of Dt', 'Transac', 'Trm Date', 'Stl Date', 'Coupon',
       'Crcy', 'Price', 'Principal', 'Accr Int', 'Settlement Amount',
       'Repo Rte Haircut', 'Unadj Term Money', 'Tkt #',
       'Execution Order Identifier'],
      dtype='object')

In [33]:
# read fee mapping, the excel have multiple sheets
df_fee = pd.read_excel('SG Fee and Comm.xlsx', sheet_name="Fee")
df_commission = pd.read_excel('SG Fee and Comm.xlsx', sheet_name="Commission")
#df_fee.head()

In [34]:
#df_commission.head()

In [35]:
# groupby blotter to mather order based on Execution Order Identifier
df_mother = df.groupby(['Trader','Counterpar','B/S','Long Description','ISIN','Trd Dt','As of Dt','Transac','Stl Date','Crcy','Execution Order Identifier']).agg({'Principal':'sum','Amount':'sum'}).reset_index()
df_mother["Average Price"] = (df_mother['Principal']/df_mother['Amount']).apply(lambda x: traditional_round(x, 4))
df_mother["Gross Amount"] = df_mother['Amount'] * df_mother['Average Price']
df_mother.loc[df_mother['Crcy'] == 'JP', 'Gross Amount'] = df_mother[df_mother['Crcy'] == 'JP']['Gross Amount'].apply(lambda x: traditional_round(x, 0))
# calculate commission
df_mother = df_mother.merge(df_commission,left_on=['Counterpar','Crcy'], right_on=['Counterparty','CURRENCY'],how='left')
df_mother = df_mother.rename(columns={'Counterparty':'CM_Counterparty','CURRENCY':'CM_Currency','FEE_CHARGE':'CM_fee','Min':'CM_min'})
df_mother['CM_fee'] = df_mother['CM_fee'].fillna(0)
df_mother['Commission'] = df_mother['CM_fee'] * df_mother['Gross Amount']
df_mother.loc[df_mother['Commission'] < df_mother['CM_min'], 'Commission'] = df_mother['CM_min']
df_mother.drop(columns=['CM_Counterparty','CM_Currency','CM_fee','CM_min'], inplace=True)
df_mother['Commission'] = np.where(df_mother['Crcy'] == 'JPY', 
                                   df_mother['Commission'].apply(lambda x: traditional_round(x, 0)), 
                                   df_mother['Commission'].apply(lambda x: traditional_round(x, 2)))
# calculate all fee
df_mother['Sec Fee'] = 0
df_mother['Stamp Duty'] = 0
df_mother['Transaction Levy'] = 0
df_mother['Trading Fee'] = 0
df_mother['AFRC Transaction Levy'] = 0
for i, row in df_mother.iterrows():
    currency = row['Crcy']
    operation = row['B/S']
    principle = row['Gross Amount']
    df_fee_schema = df_fee[(df_fee['Currency'] == currency) & (df_fee['Operations'].str[:1] == operation)]
    for index, fee_row in df_fee_schema.iterrows():
        charge_name = fee_row['Charges Name']
        charge_percentage = fee_row['Value']
        charge_round_method = fee_row['Round Method']
        charge_decimal = fee_row['Decimal']
        charge_min = fee_row['Maximum Charges']
        cost = principle * charge_percentage * 0.01
        if charge_round_method == 'Round Up':
            multiplier = 10 ** charge_decimal
            cost = math.ceil(cost * multiplier) / multiplier
        else:
            cost = traditional_round(cost, 2)
        if cost < charge_min:
            cost = charge_min
        df_mother.loc[i, charge_name] = cost
df_mother
# calculate final settlement amount
def calculate_settlement_amount(row):
    if row['B/S'] == "B":
        return row['Gross Amount'] + row['Commission'] + row['Sec Fee'] + row['Stamp Duty'] + row['Transaction Levy'] + row['Trading Fee'] + row['AFRC Transaction Levy']
    else:
        return row['Gross Amount'] - row['Commission'] - row['Sec Fee'] - row['Stamp Duty'] - row['Transaction Levy'] - row['Trading Fee'] - row['AFRC Transaction Levy']
df_mother['Settlement Amount'] = df_mother.apply(calculate_settlement_amount, axis=1)
df_mother

,Trader,Counterpar,B/S,Long Description,ISIN,Trd Dt,As of Dt,Transac,Stl Date,Crcy,...,Amount,Average Price,Gross Amount,Commission,Sec Fee,Stamp Duty,Transaction Levy,Trading Fee,AFRC Transaction Levy,Settlement Amount
0,GMSP5,GMSP5BL,B,1024,KYG532631028,02/13/25,02/13/25,02/13/25,02/17/25,HKD,...,20000,49.1785,983570.00,0.00,0,984,26.56,55.57,1.48,984637.61
1,GMSP5,GMSP5BL,B,1211,CNE100000296,02/13/25,02/13/25,02/13/25,02/17/25,HKD,...,4000,350.0750,1400300.00,0.00,0,1401,37.81,79.12,2.10,1401820.03
2,GMSP5,GMSP5BL,B,2015,KYG5479M1050,02/13/25,02/13/25,02/13/25,02/17/25,HKD,...,4300,100.0047,430020.21,0.00,0,431,11.61,24.30,0.65,430487.77
3,GMSP5,GMSP5BL,B,9866,KYG6525F1028,02/13/25,02/13/25,02/13/25,02/17/25,HKD,...,30000,33.1080,993240.00,0.00,0,994,26.82,56.12,1.49,994318.43
4,GMSP5,GMSP5BL,B,9888,KYG070341048,02/13/25,02/13/25,02/13/25,02/17/25,HKD,...,10000,93.6000,936000.00,0.00,0,936,25.27,52.88,1.40,937015.55
5,GMSP5,GMSP5BL,S,7203,JP3633400001,02/13/25,02/13/25,02/13/25,02/17/25,JPY,...,10000,2849.1050,28491050.00,0.00,0,0,0.00,0.00,0.00,28491050.00
6,GMSP5,GMSP5BL,S,9868,KYG982AW1003,02/13/25,02/13/25,02/13/25,02/17/25,HKD,...,13600,62.3443,847882.48,0.00,0,848,22.89,47.91,1.27,846962.41
7,GMSP5,HKEX,B,858,CNE000000VQ8,02/13/25,02/13/25,02/13/25,02/13/25,CNY,...,10000,131.6732,1316732.00,0.00,0,0,35.55,74.40,1.98,1316843.93
8,GMSP5,HKEX,B,300124,CNE100000V46,02/13/25,02/13/25,02/13/25,02/13/25,CNY,...,20000,65.3428,1306856.00,0.00,0,0,35.29,73.84,1.96,1306967.09
9,GMSP5,HKEX,B,300750,CNE100003662,02/13/25,02/13/25,02/13/25,02/13/25,CNY,...,2000,262.3190,524638.00,0.00,0,0,14.17,29.64,0.79,524682.60


In [36]:
#adjust output format
df_mother.rename(columns={'Counterpar':'Counterparty','Long Description':'Security Description','Trd Dt':'Trade Date','Transac':'Transaction Date','Stl Date':'Settle Date','Crcy':'Currency'},inplace=True)

In [59]:
trade_date = datetime.strptime(df_mother['Trade Date'][0], "%m/%d/%y").strftime('%Y%m%d')
file_name = "C:\\Users\\jennychen\\Desktop\\pnl_automation_project\\daily_blotter\\output\\toms_trade_blotter_{}.xlsx".format(trade_date) 
#df_mother.to_excel(file_name, index=False)

In [60]:
wb = Workbook()
ws = wb.active
for r_idx, row in enumerate(dataframe_to_rows(df_mother, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx, column=c_idx, value=value)
header_fill = PatternFill(start_color='ADD8E6', end_color='ADD8E6', fill_type='solid')
for cell in ws[1]:
    cell.fill = header_fill
    
for column in ws.columns:
    max_length = 0
    column_letter = column[0].column_letter
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    adjusted_width = (max_length + 2)  
    ws.column_dimensions[column_letter].width = adjusted_width

wb.save(file_name)
print(f"文件已保存为 {file_name}")

文件已保存为 C:\Users\jennychen\Desktop\pnl_automation_project\daily_blotter\output\toms_trade_blotter_20250213.xlsx


In [66]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders

# 邮件配置
smtp_server = 'outlook.office365.com'
sender_email = 'jennychen@tfisec.com'
sender_password = 'Cmd1998122!'
receiver_email = 'jennychen@tfisec.com'

# 创建邮件对象
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = receiver_email
message['Subject'] = 'Toms Trade Blotter on ' + f"{trade_date}"

# 添加邮件正文
body = 'Please see the attachment for toms trade blotter on ' + f"{trade_date}." + "\n\n"
message.attach(MIMEText(body, 'plain'))

# 添加附件
attachment_path = file_name
attachment = open(attachment_path, 'rb')

part = MIMEBase('application', 'octet-stream')
part.set_payload((attachment).read())
encoders.encode_base64(part)

file_name_ = f"toms_trade_blotter_{trade_date}.xlsx" 
part.add_header('Content-Disposition', f"attachment; filename= {file_name_}")

message.attach(part)

# 连接到 SMTP 服务器并发送邮件
server = smtplib.SMTP(smtp_server, 587)
server.starttls()
server.login(sender_email, sender_password)
text = message.as_string()
server.sendmail(sender_email, receiver_email, text)
server.quit()
attachment.close()